In [14]:
import os
import numpy as np
import math
import sys
import pandas as pd
from sklearn.impute import KNNImputer
import pytz
import seaborn as sns
import csv
from scipy.stats import skew, boxcox
from math import log, sqrt
import pickle
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import re
plt.rcParams['figure.facecolor'] = 'white'

In [15]:
np.set_printoptions(threshold=sys.maxsize, suppress=True)
pd.set_option('display.max_rows', None) # to show all rows in a Series
pd.set_option('display.max_columns', None) # to show all columns in a DataFrame
pd.set_option('display.width', None) # to let pandas set the display width
pd.set_option('display.max_colwidth', -1) # to display the full contents of each column

<ipython-input-15-4799ce74be0c>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) # to display the full contents of each column


In [16]:
os.chdir('C:/Users/Mario/OneDrive - Tartu Ülikool/IT_mitteinformaatikutele_MSc/Kevad 2023/Student_Project_Contest_UT_2023/Code')
df = pd.read_csv("2002-2022_encoded_csv.csv", encoding='cp1252')

In [17]:
FIGURE_PATH = os.path.join(os.getcwd(), "figures")
if not os.path.exists(FIGURE_PATH):
    os.makedirs(FIGURE_PATH)
plt.rcParams['savefig.directory'] = FIGURE_PATH

In [18]:
# Translating the names of the columns to english
df.rename({'liiklusonnetus': 'is_traffic_accident', 'toovoimetuspaevi': 'lost_days', 'kokkupuute_vigastuse_laadi_materiaalne_mojur': 'material_agent_of_contact_mode', 'kokkupuude_vigastuse_laad': 'contact_mode_of_injury', 'korvalekalde_materiaalne_mojur': 'material_agent_of_deviation', 'korvalekalle': 'deviation', 'tegevuse_materiaalne_mojur': 'material_agent_of_physical_act.', 'konkreetne_fuusiline_tegevus': 'specific_physical_activity', 'tooprotsess': 'working_process', 'tookeskkond': 'working_environment', 'tootamiskoht': 'workstation', 'vigastatud_kehaosa': 'injured_bodypart', 'vigastuse_liik': 'type_of_injury', 'ettevotte_tootajate_arv_kood': 'enterprise_size', 'vanusegrupp': 'age_group', 'otsus_uurimiseks': 'under_investigation', 'Tooandja ID': 'enterprise_ID', 'Isiku ID': 'employee_ID','ametikood': 'profession_code', 'tooalane_seisund': 'employment_status', 'pohjus': 'causes', 'pohjused_UKV': 'causes_verified', 'ettevotte_tootajate_arv': 'employees_in_enterprise', 'tootajate_arv_struktuuriyksuses': 'employees_in_structural_unit', 'onnetuse_tegevusala': 'business_area', 'sugu': 'sex', 'vanus': 'age', 'toostaa_selles_ametis_selle_tooandja_juures': 'employment_years', 'onnetuse_kellaaeg': 'time', 'eestisse_lahetatud_tootaja': 'is_posted_worker', 'onnetuse_kuupaev': 'date', 'taistunnid_paeva_vahetuse_algusest': 'full_hours_from_startofwork', 'raskusaste': 'severity', 'rap_tookeskk_riskianalyys_tehtud': 'is_risk_assessment_done', 'rap_kas_riskid_arvestatud':  'are_risks_considered', 'kodakondsus': 'citizenship', 'toimumiskoht': 'location'}, axis=1, inplace=True)

In [19]:
# Clean dataset
cols = df.columns

for i in cols:
    df.loc[df[i] == ' -- ', i] = np.nan
    df.loc[df[i] == ',, -- ', i] = np.nan
    df.loc[df[i] == ' ', i] = np.nan 
    df.loc[df[i] == '09-May', i] = np.nan 

In [20]:
# select the rows where at least 8 features are missing
rows_with_14_nan = df[df.isna().sum(axis=1) >= 8]

# drop the rows with at least 8 NaN values from the dataframe
df = df.drop(rows_with_14_nan.index)

In [21]:
df = df.drop(columns=['enterprise_ID', 'employee_ID', 'under_investigation', 'causes_verified', 'is_traffic_accident', 'is_posted_worker', 'employees_in_structural_unit', 'is_risk_assessment_done', 'are_risks_considered'])

In [22]:
def enterprise_size(string):
    if type(string) != float: 
        code = string.split(' -- ')
        if code[0] == '':
            try:
                if code[1][0] == '0':
                    return '0'
            except:
                return np.nan
        elif code[0] == '9':
            return np.nan
        else:
            return code[0]
    else:
        return np.nan


df['enterprise_size'] = df['enterprise_size'].apply(lambda x: enterprise_size(x))

In [23]:
df = df.dropna(subset=['enterprise_size'])

In [24]:
ordinal_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '0': 0
}

df['enterprise_size_ordinal_enc'] = df['enterprise_size'].map(ordinal_mapping)
df['enterprise_size_ordinal_enc'] = df['enterprise_size_ordinal_enc'].astype('float64')
df.drop(columns=['enterprise_size', 'employees_in_enterprise'], inplace=True)

In [25]:
# Function to consolidate the sex codes
def business_area(string):
    if type(string) != float: 
        code = string.split(' -- ')
        if code[0] == '':
            return '00'
        else:
            return code[0][0:2]
    else:
        return '00'


df['business_area'] = df['business_area'].apply(lambda x: business_area(x))

In [26]:
def add_general_business_code(df1, df2):
    # replace non-numeric values in 'business_area' column with NaN
    df1['business_area'] = pd.to_numeric(df1['business_area'], errors='coerce')
    
    # create a dictionary mapping business area codes to general business codes
    code_map = df2.set_index('Kood')['Üldine_tähis'].to_dict()
    
    # add a new 'General_business_code' column to df1, initially filled with NaN
    df1['business_sector'] = pd.Series(dtype='float64')
    
    # iterate over the rows in df1, checking if the business area code is in the dictionary
    for i, row in df1.iterrows():
        business_area_code = row['business_area']
        if np.isnan(business_area_code):
            general_code = None
        elif business_area_code in code_map:
            general_code = code_map[business_area_code]
        else:
            general_code = None
        df1.at[i, 'business_sector'] = general_code
    
    return df1

df_csv = pd.read_csv('emtak.csv', sep=';')
df = add_general_business_code(df, df_csv)

In [27]:
# Function to consolidate the sex codes
def sex(string):
    if type(string) != float: 
        code = string.split(' -- ')
        if code[0] == '':
            return '9'
        else:
            return code[0]
    else:
        return '9'


df['sex'] = df['sex'].apply(lambda x: sex(x))

In [28]:
ordinal_mapping = {
    '010 -- 0-17': 1,
    '1 -- 18-24': 2,
    '2 -- 25-34': 3,
    '3 -- 35-44': 4,
    '4 -- 45-54': 5,
    '5 -- 55-64': 6,
    '6 -- 65 või vanem': 7

}

df['age_group'] = df['age_group'].map(ordinal_mapping)

In [29]:
df = df.drop(columns=['age_group'])

In [30]:
# Function to consolidate the sex codes
def employment_status(string):
    if type(string) != float: 
        code = string.split(' -- ')
        if code[0] == '':
            return '000'
        else:
            return code[0]
    else:
        return '000'


df['employment_status'] = df['employment_status'].apply(lambda x: employment_status(x))

In [31]:
df = df[df['employment_status'] != '000']
df.loc[df['employment_years'] > 2000, 'employment_years'] = np.nan
subset = df.loc[(df['employment_years'] > 0) & (df['employment_years'] < 1), 'employment_years']
median = subset.median()
df['employment_years'] = df['employment_years'].replace(0, median)

In [32]:
df.dropna(subset='time', inplace=True)

In [33]:
def fix_time(time_str):
    if len(time_str) < 4:
        time_str = '00:00'
    else:
        time_str = time_str[0:2]+':00'
    return time_str

df['time'] = df['time'].apply(lambda x: fix_time(x))
df['datetime'] = df['date'] + 'T' + df['time']

In [34]:
# define the input format for the string datetime values
input_format = '%d-%m-%yT%H:%M'

# create a function to convert datetime strings to UTC timezone
def convert_to_utc(string_datetime):
    # parse the string datetime into a datetime object
    dt = datetime.datetime.strptime(string_datetime, input_format)
    
    # create a timezone object for the local time zone
    local_tz = pytz.timezone('Europe/Istanbul') # for UTC+3
    
    # set the timezone of the datetime object to the local time zone
    dt_local = local_tz.localize(dt)
    
    # convert the datetime object to UTC timezone
    utc_tz = pytz.utc
    dt_utc = dt_local.astimezone(utc_tz)
    
    # format the UTC datetime as a string
    output_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    return dt_utc.strftime(output_format)

# apply the convert_to_utc function to the datetime column in your DataFrame
df['datetime'] = df['datetime'].apply(convert_to_utc)

In [35]:
df['datetime'] = pd.to_datetime(df['datetime']).apply(lambda x: x.strftime('%Y-%m-%dT%H:%M'))

In [36]:
def create_time_features(df, date_column_name):
    # Check if date_column_name is a datetime-like object
    if not pd.api.types.is_datetime64_dtype(df[date_column_name]):
        df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce')
        if not pd.api.types.is_datetime64_dtype(df[date_column_name]):
            raise ValueError("Column {} cannot be converted to datetime.".format(date_column_name))

    # Check for missing or invalid values in the date_column_name column
    if df[date_column_name].isnull().values.any():
        raise ValueError("Column {} contains missing values.".format(date_column_name))
    
    # Create time features
    df = df.copy()
    df['dayofweek'] = (df[date_column_name].dt.dayofweek+1).astype(str) # Day of week 1..7
    df['month'] = df[date_column_name].dt.month.astype(str) # Month 1..12
    return df

df = create_time_features(df, 'date')

In [37]:
df = df.dropna(subset=['time'])

In [38]:
def time(x):
    if ':' not in x:
        return '99'
    try:
        s = x.split(':')
        if s[0] == '':
            return '00'
        else:
            return s[0]
    except:
        return '99'
    
df['time_ESAW'] = df['time'].apply(lambda x: time(x))

# Convert 'time' to numeric format
df['time_int'] = df['time_ESAW'].astype(int)

# Convert 'time' to 'sin_time' and 'cos_time'
df['sin_time'] = np.sin(2*np.pi*df['time_int']/24)
df['cos_time'] = np.cos(2*np.pi*df['time_int']/24)


In [39]:
df = df.drop(columns='time_int')

In [40]:
df['is_business_hour'] = df['time'].apply(lambda x: 1 if '08' <= x <= '16' else 0)

In [41]:
df = df.dropna(subset=['full_hours_from_startofwork'])

In [42]:
constant = 0.5
df['full_hours_from_startofwork'] = df['full_hours_from_startofwork'].replace(0, constant)

In [43]:
# Calculate Q1 and Q3
Q1 = df['full_hours_from_startofwork'].quantile(0.25)
Q3 = df['full_hours_from_startofwork'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define upper outlier limit
upper_limit = Q3 + 1.5 * IQR

# Filter dataframe to keep only rows below upper limit
df = df[df['full_hours_from_startofwork'] <= upper_limit]

In [44]:
# assuming your feature is a pandas series
feature = df.full_hours_from_startofwork

# removing zero values if any
feature = feature.replace(0, np.nan).dropna()

# measuring skewness
skewness = skew(feature)

print("Skewness of the feature: ", skewness)

# log transform
log_feature = feature.apply(np.log)

# square root transform
sqrt_feature = feature.apply(np.sqrt)

# box-cox transform
boxcox_feature, _ = boxcox(feature)

# comparing the skewness of the transformed features
print("Skewness of log-transformed feature: ", skew(log_feature))
print("Skewness of sqrt-transformed feature: ", skew(sqrt_feature))
print("Skewness of box-cox-transformed feature: ", skew(boxcox_feature))

# choosing the best transformation based on the skewness
if abs(skew(log_feature)) < abs(skew(sqrt_feature)) and abs(skew(log_feature)) < abs(skew(boxcox_feature)):
    transformed_feature = log_feature
    print("Log transformation resulted in the lowest skewness.")
elif abs(skew(sqrt_feature)) < abs(skew(boxcox_feature)):
    transformed_feature = sqrt_feature
    print("Square root transformation resulted in the lowest skewness.")
else:
    transformed_feature = boxcox_feature
    print("Box-Cox transformation resulted in the lowest skewness.")

Skewness of the feature:  0.5173817300663335
Skewness of log-transformed feature:  -0.6641677252336851
Skewness of sqrt-transformed feature:  -0.06930141460471301
Skewness of box-cox-transformed feature:  -0.1512248475487352
Square root transformation resulted in the lowest skewness.


In [45]:
df['full_hours_from_startofwork'] = transformed_feature

In [46]:
def add_target_feature(df):
    df['target'] = df['severity'].apply(lambda x: 1 if (x == '22 -- surm' or x == '20 -- raske') else 0)
    # df = df.drop(columns='severity')
    return df

df = add_target_feature(df)

In [47]:
def transform_location(location):
    if pd.isna(location) or (type(location) == str and location.strip() == "") or location == 'Määramata':
        return np.nan
    elif type(location) == str and len(location.split()) >= 2 and location.split()[1] == 'mk':
        return location.split()[0] + ' county'
    elif location == 'Tallinn':
        return 'Harju county'
    else:
        return 'Foreign country'

df['location'] = df['location'].apply(lambda x: transform_location(x))

In [48]:
df = df.dropna(subset=['location'])

In [49]:
new_to_old_dict = {
    'Harjumaa': 'Harju county',
    'Hiiumaa': 'Hiiu county',
    'Ida-Virumaa': 'Ida-Viru county',
    'Jarvamaa': 'Järva county',
    'Jogevamaa': 'Jõgeva county',
    'Laane-Virumaa': 'Lääne-Viru county',
    'Laanemaa': 'Lääne county',
    'Parnumaa': 'Pärnu county',
    'Polvamaa': 'Põlva county',
    'Raplamaa': 'Rapla county',
    'Saaremaa': 'Saare county',
    'Tartumaa': 'Tartu county',
    'Valgamaa': 'Valga county',
    'Viljandimaa': 'Viljandi county',
    'Vorumaa': 'Võru county',
    'Foreign country': 'Foreign country'
}



In [50]:
old_to_new_dict = {value: key for key, value in new_to_old_dict.items()}
df['location'] = df['location'].replace(old_to_new_dict)

In [51]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%dT%H:%M')
df['datetime'] = df['datetime'].apply(lambda x: x.timestamp())

In [52]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

df['temperature'] = 0
df['rain'] = 0
df['snowfall'] = 0

for f, item in enumerate(list(data.keys())):
    my_dict = {}
    for key, value in data[item].items():
        if pd.isna(key):
            continue
        timestamp = key.timestamp()
        my_dict[timestamp] = value

    for i, row in df.loc[df['location'] == list(df.location.unique())[f]].iterrows():

        datetime_value = row['datetime']
        
        if datetime_value in my_dict:
            # get the list of items corresponding to the key
            items_list = my_dict[datetime_value]
            
            # set the values of the new columns in df to the corresponding items in the list
            if len(items_list) == 5:
                df.at[i, 'temperature'] = items_list[0]
                df.at[i, 'rain'] = items_list[1]
                df.at[i, 'snowfall'] = items_list[2]
            else:
                # handle unexpected list length as desired, e.g. skip or replace with default values
                continue


In [53]:
# Replace zero values with np.nan for specified rows
df.loc[df['location'] == 'Foreign country', ['temperature', 'rain', 'snowfall']] = df.loc[df['location'] == 'Foreign country', ['temperature', 'rain', 'snowfall']].replace(0, np.nan)

In [54]:
df = df.dropna(subset=['temperature', 'rain', 'snowfall'])

In [55]:
# convert string values to numeric values
df['temperature'] = pd.to_numeric(df['temperature'])
df['rain'] = pd.to_numeric(df['rain'])
df['snowfall'] = pd.to_numeric(df['snowfall'])

In [56]:
df['rain'] = np.where(df['rain'] == 0, 0, 1)

In [57]:
df['snowfall'] = np.where(df['snowfall'] == 0, 0, 1)

In [58]:
df = df.dropna(subset=['causes'])

In [59]:
def cause_list(string):
  if type(string) != float:
    m = string.split(" -- ")[0]
    if ',' in m:
      cause_list = []
      for i in m.split(','):
        if (len(i) == 3) and (i not in cause_list):
          cause_list.append(i)
      if len(cause_list) < 1:
        return np.nan
      else:
        return cause_list
    elif m == "":
        return np.nan
    elif int(m) <= 9:
        result_lst = []
        result_lst.append("00" + str(int(m)))
        return result_lst
    elif int(m) > 9:
        result_lst = []
        result_lst.append("0" + str(int(m)))
        return result_lst
    else:
      return np.nan
  else:
     return np.nan

df['causes'] = df['causes'].apply(lambda x: cause_list(x))

In [60]:
df = df.dropna(subset=['causes'])

In [61]:
# replace np.nan values with 0 in the 'causes' feature
df['causes'] = df['causes'].fillna(0)

# apply one-hot encoding on the 'causes' feature
one_hot = pd.get_dummies(df['causes'].apply(pd.Series).stack(), prefix='', prefix_sep='').max(level=0)

# modify column names of one-hot encoded features
one_hot.columns = ['cause code ' + str(col) for col in one_hot.columns]

# add the new one-hot encoded features to the original dataframe
df = pd.concat([df, one_hot], axis=1)

<ipython-input-61-3903f216591b>:5: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  one_hot = pd.get_dummies(df['causes'].apply(pd.Series).stack(), prefix='', prefix_sep='').max(level=0)


In [62]:
# Filter rows where location is 'Foreign country'
foreign_country_rows = df[df['location'] == 'Ida-Virumaa']

# Select the 'location' and 'temperature' columns
location_and_temperature = foreign_country_rows[['location', 'temperature']]

In [63]:
df = df.drop(columns='causes')

In [64]:
def citizenship(string):
    if not string:
        return '0'
    elif string.split(' -- ')[0].strip() == '':
        return '0'
    else:
        return string.split(' -- ')[0]

df['citizenship'] = df['citizenship'].fillna('').apply(citizenship)

In [65]:
df = df[df['citizenship'] != '0']

In [66]:
df = df.dropna(subset=['profession_code'])


In [67]:
# Function to consolidate the sex codes
def profession_code(string):
    if type(string) != float: 
        code = string.split(' -- ')[0]
        if code == '':
            return '00'
        else:
            return code[0:2]
    else:
        return np.nan


df['profession_code'] = df['profession_code'].apply(lambda x: profession_code(x))

In [68]:
def add_general_profession_code(df1, df2):
    # replace non-numeric values in 'business_area' column with NaN
    df1['profession_code'] = pd.to_numeric(df1['profession_code'], errors='coerce')
    
    # create a dictionary mapping business area codes to general business codes
    code_map = df2.set_index('Kood')['Üldine_tähis'].to_dict()
    
    # add a new 'General_business_code' column to df1, initially filled with NaN
    df1['general_profession_class'] = pd.Series(dtype='object')  # change dtype to 'object' for string values
    
    # iterate over the rows in df1, checking if the business area code is in the dictionary
    for i, row in df1.iterrows():
        business_area_code = row['profession_code']
        if business_area_code in code_map:
            general_code = code_map[business_area_code]
        else:
            general_code = '00'  # if not found, set as 'missing'
        df1.at[i, 'general_profession_class'] = general_code
    
    return df1

df_csv = pd.read_csv('ametikood.csv', sep=';')
df = add_general_profession_code(df, df_csv)

In [69]:
df = df[df['general_profession_class'] != 99]

In [70]:
def injury_type(string):
    if isinstance(string, str):
        code = string.split(' -- ')[0]
        if len(code) == 3:
            if (code == '999') or (code == '000'):
                return '000'
            return code[0:2] + '0'
    return np.nan
  
df['type_of_injury'] = df['type_of_injury'].apply(lambda x: injury_type(x))

In [71]:
df = df.dropna(subset=['type_of_injury'])

In [72]:
df = df[df['type_of_injury'] != '000']

In [73]:
def injured_bodypart(string):
    if isinstance(string, str):
        code = string.split(' -- ')[0]
        if len(code) == 2:
            if (code == '99') or (code == '00'):
                return '00'
            return code[0:1] + '0'
    return np.nan
  
df['injured_bodypart'] = df['injured_bodypart'].apply(lambda x: injured_bodypart(x))

In [74]:
df = df[df['injured_bodypart'] != '00']

In [75]:
def workstation(string):
    code = string.split(' -- ')[0]
    if (int(code) >= 1) and (int(code) <= 8):
        return str(int(code))
    else:
        return '0'

df['workstation'] = df['workstation'].apply(lambda x: workstation(x))

In [76]:
df = df[df['workstation'] != '0']

In [77]:
def working_environment(string):
    code = string.split(' -- ')[0]
    if (len(code) == 3):
        if (code == '999') or (code == '000'):
            return '000'
        else:
            return code[0:2] + '0'
    else:
        return np.nan

df['working_environment'] = df['working_environment'].apply(lambda x: working_environment(x))

In [78]:
df = df.dropna(subset=['working_environment'])

In [79]:
df = df[df['working_environment'] != '000']

In [80]:
def working_process(string):
    code = string.split(' -- ')[0]
    if (int(code) == 0):
        return '00'
    elif (int(code) == 13):
        return '12'
    else:
        return code[0:1] + '0'

df['working_process'] = df['working_process'].apply(lambda x: working_process(x))

In [81]:
df = df[df['working_process'] != '00']

In [82]:
df = df.dropna(subset=['deviation'])
df = df.dropna(subset=['contact_mode_of_injury'])
df = df.dropna(subset=['specific_physical_activity'])

In [83]:
def general(string):
    code = string.split(' -- ')[0]
    if (int(code) == 0):
        return '00'
    elif (int(code) == 99):
        return '00'
    else:
        return code[0:1] + '0'

df['specific_physical_activity'] = df['specific_physical_activity'].apply(lambda x: general(x))
df['deviation'] = df['deviation'].apply(lambda x: general(x))
df['contact_mode_of_injury'] = df['contact_mode_of_injury'].apply(lambda x: general(x))

In [84]:
df = df[df['specific_physical_activity'] != '00']
df = df[df['deviation'] != '00']
df = df[df['contact_mode_of_injury'] != '00']

In [85]:
df = df.rename(columns={'material_agent_of_physical_act.': 'material_agent_of_physical_act'})

In [86]:
def agent(code):
  code_part1 = code.split(' -- ')[0]
  if len(code_part1) > 2:
    code_part2 = code_part1.split('.')
    if (code_part2[0] == '00'):
      return np.nan
    else:
      return code_part2[0] + '.' + '00'
  else:
    return 'other'

df['material_agent_of_physical_act'] = df['material_agent_of_physical_act'].apply(lambda x: agent(x))
df['material_agent_of_deviation'] = df['material_agent_of_deviation'].apply(lambda x: agent(x))
df['material_agent_of_contact_mode'] = df['material_agent_of_contact_mode'].apply(lambda x: agent(x))

In [87]:
df = df[df['lost_days'] >= 0]

In [88]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [89]:
df.to_csv('data.csv', encoding='latin-1')